# Generate density split cross-correlation functions

In [1]:
import os
import tempfile
import time
import copy

import numpy as np
import scipy
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (10, 8)
plt.rcParams.update({'font.size': 12})

from pypower import CatalogMesh, MeshFFTPower, CatalogFFTPower, PowerSpectrumStatistics, utils, setup_logging
from pycorr import TwoPointCorrelationFunction, TwoPointEstimator, NaturalTwoPointEstimator, project_to_multipoles, BoxSubsampler, project_to_wp, utils, setup_logging
from abacusnbody.data.compaso_halo_catalog import CompaSOHaloCatalog
from cosmoprimo import *

import densitysplit 
from densitysplit import catalog_data, density_split
from bin.density_split_mocks_functions import split_density, compute_densitySplit_CCF
from densitysplit.pk_model import *
from densitysplit.utils import *
from densitysplit.corr_func_utils import *

## Data and output directories

In [2]:
data_dir = '/feynman/work/dphp/mp270220/data/'
output_dir = '/feynman/work/dphp/mp270220/outputs/'

## Get data

In [3]:
#catalog_name = 'AbacusSummit_1Gpc_z1.175'
catalog_name = 'AbacusSummit_2Gpc_z1.175'
#catalog_name = 'AbacusSummit_2Gpc_z0.800'
#catalog_name = 'mock'

catalog = catalog_data.Data.load(data_dir+catalog_name+'.npy')
catalog.shift_boxcenter(-catalog.offset)

## Apply RSD

In [4]:
# Abacus summit base cosmology
cosmo_abacus = fiducial.AbacusSummitBase()
bg = cosmo_abacus.get_background()
hz = 100*bg.efunc(catalog.redshift)

catalog.set_rsd(hz=hz)

## Split density

In [5]:
# Density mesh
cellsize = 10
resampler = 'tsc'
nsplits = 3

catalog_density = split_density(catalog, cellsize, resampler, nsplits, use_rsd=False, save=False)

[000001.63] [0/1] 06-16 15:26  CatalogMesh               INFO     Slab 0 ~ 4194304 / 23463294.
[000002.63] [0/1] 06-16 15:26  CatalogMesh               INFO     Painted 4194304 out of 23463294 objects to mesh.
[000002.63] [0/1] 06-16 15:26  CatalogMesh               INFO     Slab 4194304 ~ 8388608 / 23463294.
[000003.63] [0/1] 06-16 15:26  CatalogMesh               INFO     Painted 8388608 out of 23463294 objects to mesh.
[000003.63] [0/1] 06-16 15:26  CatalogMesh               INFO     Slab 8388608 ~ 12582912 / 23463294.
[000004.63] [0/1] 06-16 15:26  CatalogMesh               INFO     Painted 12582912 out of 23463294 objects to mesh.
[000004.63] [0/1] 06-16 15:26  CatalogMesh               INFO     Slab 12582912 ~ 16777216 / 23463294.
[000005.62] [0/1] 06-16 15:26  CatalogMesh               INFO     Painted 16777216 out of 23463294 objects to mesh.
[000005.62] [0/1] 06-16 15:26  CatalogMesh               INFO     Slab 16777216 ~ 20971520 / 23463294.
[000006.61] [0/1] 06-16 15:26  Cat

## Correlation galaxies-galaxies

### Correlation function in each density split

In [6]:
edges = (np.linspace(0., 150., 51), np.linspace(-1, 1, 201))
los='x'

In [7]:
results = compute_densitySplit_CCF(catalog_density, edges, los, use_rsd=False, save=True, output_dir=output_dir, name=catalog_name, randoms_size=4)

[000039.64] [0/1] 06-16 15:27  TwoPointCorrelationFunction INFO     Using estimator <class 'pycorr.twopoint_estimator.NaturalTwoPointEstimator'>.
[000039.64] [0/1] 06-16 15:27  TwoPointCorrelationFunction INFO     Computing two-point counts D1D2.
[000072.53] [0/1] 06-16 15:27  TwoPointCorrelationFunction INFO     Analytically computing two-point counts R1R2.
[000072.58] [0/1] 06-16 15:27  TwoPointCorrelationFunction INFO     Using estimator <class 'pycorr.twopoint_estimator.NaturalTwoPointEstimator'>.
[000072.58] [0/1] 06-16 15:27  TwoPointCorrelationFunction INFO     Computing two-point counts D1D2.
[000161.67] [0/1] 06-16 15:29  TwoPointCorrelationFunction INFO     Analytically computing two-point counts R1R2.
[000161.67] [0/1] 06-16 15:29  TwoPointCorrelationFunction INFO     Using estimator <class 'pycorr.twopoint_estimator.NaturalTwoPointEstimator'>.
[000161.67] [0/1] 06-16 15:29  TwoPointCorrelationFunction INFO     Computing two-point counts D1D2.
[001360.50] [0/1] 06-16 15:49  